In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sys, os
sys.path.append("./subjects/")
import time, random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from adf_data.adult import adult_data
from adf_utils.config import adult
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

from Search_utils import DT_tree
from Search_utils import causal_dataset_adult_I
from Search_utils import causal_pertubation_adult_I
from Search_utils import nodes_analysis
from Search_utils import tree_distance

from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
from sklearn.linear_model import ElasticNetCV
from lightgbm import LGBMClassifier
import shap

np.random.seed(0)
random.seed(0)
def test_cases(X_train, X_test, y_train, y_test, A_train, A_test,dataset, program_name, max_iter, 
               sensitive_param, unpriviliged_group, priviliged_group, sensitive_name):
    global df_inp, df_input_org, shap_values,fig,X, feat_imoprtance
    EOD_tresh = 0.05
    filename0 ='ShAP_NI_new'
    le =  LabelEncoder()
    #global X_train, y_train, X_test, y_test
    model = LogisticRegression()
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    acc_None = round(accuracy_score(y_test,preds),2)
    f1_None = round(f1_score(y_test,preds),2)
    print(acc_None,f1_None )
    pred_bal_None = preds.sum()/preds.shape[0]
    if(program_name == "LogisticRegression"):
        import Logistic_Regression_Mitigation
        original_program = Logistic_Regression_Mitigation.logisticRegressionOriginal
        input_program_tree = 'logistic_regression_Params.xml'
        num_args = 15
    coef_data=pd.read_csv('./Results/adult_coef1.csv')
    df_input_org = pd.read_csv('Results/'+program_name+'/'+dataset+'/'+sensitive_name+'/Inputs.csv').drop(columns=['Unnamed: 0'])
    df_input_org.columns = ['solver', 'penalty', 'dual', 'tol', 'C', 'fit_intercept', 'intercept_scaling', 
                        'max_iteration', 'multi_class', 'l1_ratio', 'class_weight', 'random_state', 'verbose',
                        'warm_start', 'n_jobs','EOD', 'time','score','intresting']
   
    cat_columns = ['solver', 'penalty', 'dual','fit_intercept', 'multi_class', 'warm_start']

    default_acc = acc_None - 0.05
    epsilon = 0.05
#-------------------------------------------------------------
    coef_list = ['e0', 'z1e', 'z2e', 'xe', 'ne', 'h0', 'z1h', 'z2h', 'xh', 'nh', 'eh',
       'w0', 'z2w', 'ew', 'nw', 'hw', 'm0', 'z1m', 'z2m', 'wm', 'hm', 'nm',
       'xm', 'o0', 'z1o', 'z2o', 'eo', 'wo', 'mo', 'xo', 'r0', 'mr', 'er',
       'z2r', 'nr', 'xr', 'y0', 'z1y', 'z2y', 'ey', 'oy', 'wy', 'my', 'hy',
       'ry', 'ny', 'xy', 'sigma_h']
#     Stratified Approach
# ----------------------------------------

#     bins = [i*0.01 for i in range(int(df_input_org['EOD'].min()*100),int(df_input_org['EOD'].max()*100)-2,1)]
#     biniazed =np.digitize(df_input_org['EOD'], bins, right=False)
#     for bin0 in bins:
#         if np.where(biniazed==bin0*100)[0].shape[0] < 5 :
#             bins = np.delete(bins,np.where(np.array(bins)==bin0)[0])
#     df_input_org['bins'] = np.digitize(df_input_org['EOD'], bins, right=False)  
#     X_train, df_inp_temp, y_train, y_test = train_test_split(df_input_org[df_input_org.columns[:15]], df_input_org['EOD'],stratify=df_input_org['bins'],test_size=0.005)
#     df_inp_temp['EOD'] = y_test
#     df_inp_temp.reset_index(drop=True, inplace=True)
    
# -------------------------------------------
# #       Same Sample size of each bin
    bins = [i*0.01 for i in range(int(df_input_org['EOD'].min()*100),int(df_input_org['EOD'].max()*100)-2,1)]
    biniazed =np.digitize(df_input_org['EOD'], bins, right=False)
    for bin0 in bins:
        if np.where(biniazed==bin0*100)[0].shape[0] < 10 :
            bins = np.delete(bins,np.where(np.array(bins)==bin0)[0])
    df_input_org['bins'] = np.digitize(df_input_org['EOD'], bins, right=False)  
    num_samples_st = 300
    num_bins = df_input_org['bins'].unique().shape[0]
    size_bins = int(num_samples_st / num_bins) + 1
    random_indices = []
    for bin0 in df_input_org['bins'].unique():
        random_indices = np.append(random_indices,np.random.choice(np.where(df_input_org['bins']==bin0)[0], size =size_bins ))
    
#         Top Down Approach        
#------------------------------------------------------
#     n_samples = 200
#     top_indices  = df_input_org.sort_values(by=['EOD']).tail(n_samples).index
#     down_indices = df_input_org.sort_values(by=['EOD']).head(n_samples).index  
#     print('label ratio' , np.where(df_input_org.iloc[np.concatenate([top_indices, down_indices])]['EOD'] >= EOD_tresh, 1, 0 ).sum()/(2*n_samples))

#------------------------------------------------------
#       dataset perturbation
    feat_imoprtance = {}
    rand_st = 0
    successful_none = False
    while successful_none != True:
        print('RS',rand_st)

        #     Stratified Approach
        #df_inp = df_inp_temp.copy(deep=True)


        #         Top Down Approach
        #df_inp = df_input_org.iloc[np.concatenate([top_indices, down_indices])].reset_index(drop=True)

        #       Same Sample size of each bin
        
        df_inp = df_input_org.iloc[random_indices].reset_index(drop=True)
        None_class_ratio = np.where(df_inp['EOD'] >= EOD_tresh, 1, 0 ).sum()/df_inp.shape[0]
        
        status_perturb, df_inp = causal_pertubation_adult_I(df,original_program, df_inp, coef_data, 
                                                           coef_list, None, 0.0 ,
                                                           False,  sensitive_name, 
                                                           default_acc=None, default_f1=None, 
                                                           random_state=rand_st, drop_feat = None)
        
        if status_perturb== False:
            print('Unable to generate dataset with this perturbation!')
            rand_st = random.randint(0,100)
            
            continue
        else:
            successful_none = True
            print('EOD of perturbing',round(df_inp['EOD2'].mean(),3) )
            labels = df_inp['EOD2']
            
            # Preprocessing the hyperparameters
            for col in cat_columns:
                df_inp[col] = le.fit_transform(df_inp[col])
            columns = []
            for col in df_inp.columns[:num_args]:
                if df_inp[col].unique().shape[0] > 1 :
                    columns.append(col)           

            df_inp = df_inp[columns]
            for ind in np.where(df_inp.isna().sum()>0):
                df_inp[df_inp.columns[ind]+'_isna']=0
                df_inp.loc[np.where(df_inp[df_inp.columns[ind]].isna()==True)[0],df_inp.columns[ind]+'_isna'] = 1
                df_inp.loc[np.where(df_inp[df_inp.columns[ind]].isna()==True)[0], df_inp.columns[ind]] = 0

            df_inp['label'] = labels

            if not os.path.exists('./Results/'):
                    os.makedirs('./Results/')
            if not os.path.exists('./Results/' + str(program_name) + '/'):
                os.makedirs('./Results/' + str(program_name) + '/')
            if not os.path.exists('./Results/' + str(program_name) + '/'+ str(dataset) + '/'):
                os.makedirs('./Results/' + str(program_name) + '/'+ str(dataset) + '/')
            if not os.path.exists('./Results/' + str(program_name) + '/'+ str(dataset) + '/' + str(sensitive_name) + '/'):
                os.makedirs('./Results/' + str(program_name) + '/' + str(dataset) + '/' + str(sensitive_name) + '/')
            if not os.path.exists('./Results/' + str(program_name) + '/'+ str(dataset) + '/' + str(sensitive_name) + '/'+filename0+'/'):
                os.makedirs('./Results/' + str(program_name) + '/' + str(dataset) + '/' + str(sensitive_name) + '/'+filename0+'/')
    #             if not os.path.exists('./Results/' + str(program_name) + '/'+ str(dataset) + '/' + str(sensitive_name) + '/'+filename0+'/'+ str(edge0) + '/'):
    #                 os.makedirs('./Results/' + str(program_name) + '/' + str(dataset) + '/' + str(sensitive_name) + '/'+filename0+'/'+ str(edge0) + '/')    

            
            dir_res = 'Results/'+program_name+'/'+dataset+'/'+sensitive_name+'/'+filename0+'/' + 'None' 
#             out = dir_res + '.dot'
#             graph = DT_tree(df_inp)
#             graph.write_png(dir_res + ".png")
#             condition = nodes_analysis(graph)
#             None_condition = condition
                        #print('Time to', causal_edge, time.time() - time1)
            X = df_inp[df_inp.columns[:-1]]
            y = df_inp['label']
            features = X.columns
            cat_features = []
            for cat in X.select_dtypes(exclude="number"):
                cat_features.append(cat)
                X[cat] = X[cat].astype("category").cat.codes.astype("category")
            clf =  ElasticNetCV(cv=10).fit(X,y,) 
            explainer = shap.LinearExplainer(clf, X,algorithm='auto', feature_perturbation="correlation_dependent")
            shap_values = explainer.shap_values(X)
            feat_imoprtance['None'] = X.columns[np.argsort(np.abs(shap_values).mean(0))]

            fig  = shap.summary_plot(shap_values, X, show=False)         
            plt.savefig(dir_res + "_shap.png")   
            fig = plt.figure()
            plt.figure().clear()
            plt.close()
            plt.cla()
            plt.clf()
            
    distance_list=[]
#     for step in range(res.shape[0]):
        
    
#         causal_edge = res['edge'][step]
#         step_size = float(res['step_size'][step])

    for stp in range(-9,9):#[0.01, -0.01]:
        if stp==0:continue
        step_size = stp/100
        for causal_edge in coef_list:
            if 'sigma' in causal_edge or '0' in causal_edge:
                continue
            time1 = time.time()

            #     Stratified Approach
            #df_inp = df_inp_temp.copy(deep=True)

            #         Top Down Approach
            #df_inp = df_input_org.iloc[np.concatenate([top_indices, down_indices])].reset_index(drop=True)

            #       Same Sample size of each bin
            df_inp = df_input_org.iloc[random_indices].reset_index(drop=True)               

            status_perturb, df_inp = causal_pertubation_adult_I(df,original_program, df_inp, coef_data, 
                                                                   coef_list, causal_edge, step_size ,
                                                                   False,  sensitive_name, 
                                                                   default_acc=None, default_f1=None, 
                                                                   random_state=rand_st, 
                                                                  drop_feat = None)

            if status_perturb== False:
                print('Unable to generate dataset with perturbing', causal_edge)
                continue
            else:
                if pd.isna(df_inp['EOD2'].mean()) == True:
                    print('Perturbation leaded low accuracy!')
                    continue
                else:
                    print('EOD of perturbing', causal_edge, df_inp['EOD2'].mean())
        #--------------------------------------------------------           

                labels = df_inp['EOD2']      
                # Preprocessing the hyperparameters
                for col in cat_columns:
                    df_inp[col] = le.fit_transform(df_inp[col])
                columns = []
                for col in df_inp.columns[:num_args]:
                    if df_inp[col].unique().shape[0] > 1 :
                        columns.append(col)           

                df_inp = df_inp[columns]
                for ind in np.where(df_inp.isna().sum()>0):
                    df_inp[df_inp.columns[ind]+'_isna']=0
                    df_inp.loc[np.where(df_inp[df_inp.columns[ind]].isna()==True)[0],df_inp.columns[ind]+'_isna'] = 1
                    df_inp.loc[np.where(df_inp[df_inp.columns[ind]].isna()==True)[0], df_inp.columns[ind]] = 0

                df_inp['label'] = labels
    #                 print('class ratio',labels.sum()/labels.shape[0])
    #                 if labels.sum() < 3 or labels.sum() > labels.shape[0] - 3 :
    #                     print('Only one class detected on ', causal_edge)
    #                     if causal_edge== None:
    #                         successful_none = True
    #                     continue

                if not os.path.exists('./Results/'):
                        os.makedirs('./Results/')
                if not os.path.exists('./Results/' + str(program_name) + '/'):
                    os.makedirs('./Results/' + str(program_name) + '/')
                if not os.path.exists('./Results/' + str(program_name) + '/'+ str(dataset) + '/'):
                    os.makedirs('./Results/' + str(program_name) + '/'+ str(dataset) + '/')
                if not os.path.exists('./Results/' + str(program_name) + '/'+ str(dataset) + '/' + str(sensitive_name) + '/'):
                    os.makedirs('./Results/' + str(program_name) + '/' + str(dataset) + '/' + str(sensitive_name) + '/')
                if not os.path.exists('./Results/' + str(program_name) + '/'+ str(dataset) + '/' + str(sensitive_name) + '/'+filename0+'/'):
                    os.makedirs('./Results/' + str(program_name) + '/' + str(dataset) + '/' + str(sensitive_name) + '/'+filename0+'/')
        #             if not os.path.exists('./Results/' + str(program_name) + '/'+ str(dataset) + '/' + str(sensitive_name) + '/'+filename0+'/'+ str(edge0) + '/'):
        #                 os.makedirs('./Results/' + str(program_name) + '/' + str(dataset) + '/' + str(sensitive_name) + '/'+filename0+'/'+ str(edge0) + '/')    

                dir_res = 'Results/'+program_name+'/'+dataset+'/'+sensitive_name+'/'+filename0+'/' + str(causal_edge)  + ('_pos_' if step_size >0 else '_neg_') + str(abs(step_size))
    #                 out = dir_res + '.dot'
    #                 graph = DT_tree(df_inp)
    #                 graph.write_png(dir_res + ".png")
    #                 condition = nodes_analysis(graph)
    #                 if step_size > 0 : 
    #                     distance_pos[causal_edge] = tree_distance(None_condition, condition)
    #                 else :
    #                     distance_neg[causal_edge] = tree_distance(None_condition, condition)
                    #print('Time to this edge',time.time() - time1)
                X = df_inp[df_inp.columns[:-1]]
                y = df_inp["label"]
                features = X.columns
                cat_features = []
                for cat in X.select_dtypes(exclude="number"):
                    cat_features.append(cat)
                    X[cat] = X[cat].astype("category").cat.codes.astype("category")
                clf =  ElasticNetCV(cv=10).fit(X,y,) 
                explainer = shap.LinearExplainer(clf, X,algorithm='auto', feature_perturbation="correlation_dependent")
                shap_values = explainer.shap_values(X)
                feat_imoprtance[causal_edge+'_'+ str(step_size)] = X.columns[np.argsort(np.abs(shap_values).mean(0))]
                distance = 0 
                for feat in  feat_imoprtance[causal_edge+'_'+ str(step_size)][-4:]:
                    if feat not in feat_imoprtance['None'][-4:]:
                        distance+=1
                distance_list.append(distance)      
                print(causal_edge,' feature importance')
                fig  = shap.summary_plot(shap_values, X, show=False)
                plt.savefig(dir_res +"_shap.png")   
                fig = plt.figure()
                plt.figure().clear()
                plt.close()
                plt.cla()
                plt.clf()
    #res['dis'] = distance_list
    #res.to_csv('Results/'+program_name+'/'+dataset+'/'+sensitive_name+'/distances.csv')
#     np.save('Results/'+program_name+'/'+dataset+'/'+sensitive_name+'/'+filename0+'/png/' + str(causal_edge) + '_pos.npy' ,distance_pos)
#     np.save('Results/'+program_name+'/'+dataset+'/'+sensitive_name+'/'+filename0+'/png/' + str(causal_edge) + '_neg.npy' ,distance_neg)
#     print('Negative' ,distance_neg)
#     print('Postive' ,distance_pos)
if __name__ == '__main__':
    num_iteration =  100000
    program_name="LogisticRegression"
    data = {'adult':adult_data}
    data_config = {"adult":adult}
    for dataset in data_config.keys(): 
        if dataset == "adult":
            sens_list =[8]
        for sensitive_param in sens_list:
            if dataset == "adult" and sensitive_param == 8:
                sensitive_name = "gender"
                priviliged_group = [{sensitive_name: 1.0}]  #male
                unpriviliged_group = [{sensitive_name: 0.0}]  #female
                favorable_label  = 1.0
                unfavorable_label = 0.0
            X, Y, input_shape, nb_classes = data[dataset]()

            df = pd.DataFrame(X)
            df.columns = data_config[dataset]().feature_name
            A = df[sensitive_name].to_numpy()
            X = df.to_numpy()

            df['label'] = Y

            X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(X, Y, A, random_state=0,stratify=Y)
            
            test_cases(X_train, X_test, y_train, y_test, A_train, A_test, dataset, program_name, num_iteration, 
                       sensitive_param, unpriviliged_group, priviliged_group, sensitive_name)